# Analyzing the results for `lincs_rdkit_hparam`

In [1]:
from pathlib import Path
import matplotlib
import pandas as pd
import seaborn as sn
import seml
from matplotlib import pyplot as plt

matplotlib.style.use("fivethirtyeight")
matplotlib.style.use("seaborn-talk")
matplotlib.rcParams['font.family'] = "monospace"
plt.rcParams['savefig.facecolor'] = 'white'
sn.set_context("poster")

In [2]:
results = seml.get_results(
    "lincs_rdkit_hparam",
    to_data_frame=True,
    fields=["config", "result", "seml", "config_hash"],
    states=["COMPLETED"],
)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

/home/simon/miniconda3/envs/chemical_CPA/lib/python3.7/site-packages/seml/evaluation.py:80: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  parsed = pd.io.json.json_normalize(parsed, sep='.')


In [3]:
results

,_id,config_hash,seml.executable,seml.name,seml.output_dir,seml.conda_environment,seml.working_dir,seml.source_files,seml.output_file,seml.command,...,result.penalty_adv_covariates,result.elapsed_time_min,result.training,result.test,result.ood,result.perturbation disentanglement,result.optimal for perturbations,result.covariate disentanglement,result.optimal for covariates,result.total_epochs
0,1,951c6cef0caf7660d98b49c30a3cb0ca,compert/seml_sweep_icb.py,lincs_rdkit_hparam,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,chemical_CPA,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,"[[compert/data.py, 61e6ea51e98f7c50f0d48fcd], ...",/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,python compert/seml_sweep_icb.py with 'profili...,...,"[12.919138175999706, 0.18419641748869253, 0.17...",347.128484,"[[0.9075990746769373, 0.7284160405249093, -0.0...","[[0.8994284446040789, 0.7162557100256284, -0.1...","[[0.9028372780552932, 0.7350117628063474, -0.1...",[0.2929533123970032],[0.006978477740242026],[[0.8103122711181641]],[[0.012195121951219513]],125
1,2,77032057a974ed92ba42eb00b4818b39,compert/seml_sweep_icb.py,lincs_rdkit_hparam,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,chemical_CPA,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,"[[compert/data.py, 61e6ea51e98f7c50f0d48fcd], ...",/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,python compert/seml_sweep_icb.py with 'profili...,...,"[8.224835022303726, 0.10176633771588237, 0.049...",716.408253,"[[0.8747260873871502, 0.5969719429824801, -0.0...","[[0.8460550049940745, 0.5034949233134588, -0.0...","[[0.8460868437375341, 0.4970656345997538, -0.1...",[0.24740289151668549],[0.006978477740242026],[[0.629379391670227]],[[0.012195121951219513]],150
2,3,10b608acc528830eb6412e70557b6999,compert/seml_sweep_icb.py,lincs_rdkit_hparam,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,chemical_CPA,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,"[[compert/data.py, 61e6ea51e98f7c50f0d48fcd], ...",/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,python compert/seml_sweep_icb.py with 'profili...,...,"[3.442848868081695, 0.12761329248833608, 0.081...",371.785198,"[[0.9078908162748265, 0.7433778024650205, -0.0...","[[0.9045214022199313, 0.749026610950629, -0.05...","[[0.908264586435897, 0.7642039332006659, -0.08...",[0.04496359825134277],[0.006978477740242026],[[0.6037176251411438]],[[0.012195121951219513]],125
3,4,1891f3ccb3ec6f0369e6083027911835,compert/seml_sweep_icb.py,lincs_rdkit_hparam,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,chemical_CPA,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,"[[compert/data.py, 61e6ea51e98f7c50f0d48fcd], ...",/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,python compert/seml_sweep_icb.py with 'profili...,...,"[6.160079438468074, 0.08430342018991865, 0.156...",281.124688,"[[0.8941680782485102, 0.6784092028443669, -0.0...","[[0.8730832556883494, 0.626161656777064, -0.07...","[[0.8751987394477639, 0.6370474488607475, -0.1...",[0.2513996362686157],[0.006978477740242026],[[0.6995764970779419]],[[0.012195121951219513]],125
4,5,e1176dbeeb762c548b23bbb5c1ea60d9,compert/seml_sweep_icb.py,lincs_rdkit_hparam,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,chemical_CPA,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,"[[compert/data.py, 61e6ea51e98f7c50f0d48fcd], ...",/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,python compert/seml_sweep_icb.py with 'profili...,...,"[24.995399655632355, 0.7440850614839292, 1.031...",278.147404,"[[0.9054807058799824, 0.721111265594224, -0.04...","[[0.8911898617943128, 0.6951551422476768, -0.0...","[[0.8944398718220847, 0.7103013859263488, -0.1...",[0.2972831428050995],[0.006978477740242026],[[0.9510713219642639]],[[0.012195121951219513]],125
5,8,5aae0f7165b514905025dbfdad4cf8eb,compert/seml_sweep_icb.py,lincs_rdkit_hparam,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,chemical_CPA,/mnt/home/icb/simon.boehm/Masters_thesis/chemi...,"[[compert/data.py, 61e6ea51e98f7c50f0d48fcd], ...",/mnt/home/icb/simon.boehm/